In [1]:
from HTMLParser import HTMLParser
import numpy as np
from __future__ import division
import vincent
import nltk
from nltk.corpus import stopwords
from nltk import FreqDist
import pandas as pd

In [2]:
# Load the script
with open('script_LOTRIII.html', 'r') as myfile:
    script=myfile.read().replace('\n', '')

# Modify the script to fix some issues.
script = script.replace(" O.S", "-")
script = script.replace("G AN DA LF", "GANDALF")
script = script.replace("CANDALF", "GANDALF")
script = script.replace("WITCH KING", "WITCH-KING")

In [3]:
# Create an HTML Parser to get the characters.
class MyHTMLCharacterParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.is_b = False
        self.characters = set()
    
    def handle_starttag(self, tag, attrs):
        if tag == "b":
            self.is_b = True

    def handle_endtag(self, tag):
        if tag == "b":
            self.is_b = False

    def handle_data(self, data):
        if self.is_b == True:
            if data[0] == " " and ":" not in data and "(" not in data:
                self.characters.add(data.strip(" ,.·;:\\\"/-)"))
                

# Instantiate the parser and feed it the script.
parser = MyHTMLCharacterParser()
parser.feed(script)

# Clean the character list.
parser.characters.remove("A")
parser.characters.remove("CUT")
parser.characters.remove("AE")
parser.characters.remove("L")
parser.characters.remove("N")
parser.characters.remove("'")
parser.characters.remove("IG")
parser.characters.remove("CUT TO")
parser.characters.remove("FADE TO BLACK")
parser.characters.remove("THE END")
parser.characters.remove("")

for elem in list(parser.characters):
    if "V/0" in elem:
        parser.characters.remove(elem)

print len(parser.characters)

characters = []
for elem in parser.characters:
    characters.append(elem)
    print elem

43
MADRIL
GALADRIEL
SMEAGOL
KING OF THE DEAD
ORCS
SOLDIER
DEAGOL
GIMLI
GRIMBOLD
SOLDIER 2
LEGOLAS
FARAMIR
SAM
EOMER
BILBO
IORLAS
DENETHOR
ARWEN
FRODO
GUARD
SOLDIER 1
GOTHMOG
LIEUTENANT
ORC 2
TREEBEARD
ORC COMMANDER
PIPPIN
GORBAG
ROHIRRIM
WITCH-KING
ROHAN SOLDIER
GAMLING
ELF ESCORT
THEODEN
EOWYN
GATE GUARD
RANGER
GOLLUM
MERRY
GANDALF
ARAGORN
ELROND
SHAGRAT


In [4]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()

# Add the nodes
for char in characters:
    G.add_node(char)

In [5]:
# Add the edges, find the scenes using the words "EXT." and "INT."
class MyHTMLSceneParser(HTMLParser):
    def __init__(self, characters):
        HTMLParser.__init__(self)
        self.is_b = False
        self.characters = characters
        self.scenes = []
        self.char_in_scene = set()
    
    def handle_starttag(self, tag, attrs):
        #print "Encountered a start tag:", tag
        if tag == "b":
            self.is_b = True
        
    def handle_endtag(self, tag):
        #print "Encountered an end tag :", tag
        if tag == "b":
            self.is_b = False
        
    def handle_data(self, data):
        #print "Encountered some data  :", data
        if data[:4] == "EXT." or data[:4] == "INT.":
            self.scenes.append(self.char_in_scene)
            self.char_in_scene = set()
        else:
            if self.is_b:
                char = data.strip(" ,.·;:\\\"/-)")
                if char in self.characters:
                    self.char_in_scene.add(char)
                
                

# Instantiate the parser and fed it the script.
parser = MyHTMLSceneParser(characters)
parser.feed(script)
# Add the last scene.
parser.scenes.append(parser.char_in_scene)

all_scenes = parser.scenes


In [6]:
# Add the edges to the graph.

for scene in parser.scenes:
    for i, c in enumerate(list(scene)):
        for j in range(i+1, len(scene)):
            if G.has_edge(c, list(scene)[j]):
                G[c][list(scene)[j]]['weight'] = G[c][list(scene)[j]]['weight'] + 1
            else:
                G.add_edge(c, list(scene)[j], weight=1)



In [7]:
# Remove the nodes with degree 0.
for char in characters:
    if G.degree(char) == 0:
        G.remove_node(char)
        print char


ORCS
GUARD
ELF ESCORT


In [8]:
x = nx.algorithms.pagerank(G)

import operator
sorted_x = sorted(x.items(), key=operator.itemgetter(1))
sorted_x.reverse()
sorted_x[:10]

[('GANDALF', 0.08938728122441608),
 ('PIPPIN', 0.07299706063732403),
 ('THEODEN', 0.06436694958276629),
 ('FRODO', 0.057627052983407694),
 ('SAM', 0.05442678981274595),
 ('ARAGORN', 0.053588076862994355),
 ('MERRY', 0.049913034808897874),
 ('GIMLI', 0.04665746599568066),
 ('EOWYN', 0.04398983986203521),
 ('GOLLUM', 0.03530083263275775)]

In [9]:
import community

com = community.best_partition(G)

sorted_x = sorted(com.items(), key=operator.itemgetter(1))
sorted_x.reverse()
sorted_x

[('GANDALF', 3),
 ('SOLDIER', 3),
 ('MERRY', 3),
 ('GATE GUARD', 3),
 ('PIPPIN', 3),
 ('TREEBEARD', 3),
 ('DENETHOR', 3),
 ('IORLAS', 3),
 ('SOLDIER 1', 3),
 ('SOLDIER 2', 3),
 ('ARAGORN', 2),
 ('EOWYN', 2),
 ('THEODEN', 2),
 ('GAMLING', 2),
 ('ROHAN SOLDIER', 2),
 ('WITCH-KING', 2),
 ('ROHIRRIM', 2),
 ('LEGOLAS', 2),
 ('EOMER', 2),
 ('GRIMBOLD', 2),
 ('GIMLI', 2),
 ('KING OF THE DEAD', 2),
 ('SHAGRAT', 1),
 ('ELROND', 1),
 ('DEAGOL', 1),
 ('GOLLUM', 1),
 ('GORBAG', 1),
 ('ORC 2', 1),
 ('ARWEN', 1),
 ('BILBO', 1),
 ('FRODO', 1),
 ('SAM', 1),
 ('SMEAGOL', 1),
 ('GALADRIEL', 1),
 ('LIEUTENANT', 0),
 ('ORC COMMANDER', 0),
 ('GOTHMOG', 0),
 ('FARAMIR', 0),
 ('RANGER', 0),
 ('MADRIL', 0)]

In [10]:
x = nx.algorithms.degree_centrality(G)

sorted_x = sorted(x.items(), key=operator.itemgetter(1))
sorted_x.reverse()
sorted_x[:10]

[('GANDALF', 0.5128205128205128),
 ('THEODEN', 0.3846153846153846),
 ('GIMLI', 0.3076923076923077),
 ('PIPPIN', 0.28205128205128205),
 ('MERRY', 0.28205128205128205),
 ('ARAGORN', 0.2564102564102564),
 ('EOWYN', 0.2564102564102564),
 ('GOTHMOG', 0.2564102564102564),
 ('SAM', 0.2564102564102564),
 ('ELROND', 0.20512820512820512)]

In [11]:
x = nx.algorithms.betweenness_centrality(G)

sorted_x = sorted(x.items(), key=operator.itemgetter(1))
sorted_x.reverse()
sorted_x[:10]

[('GANDALF', 0.48792172739541173),
 ('THEODEN', 0.2018443544759334),
 ('SAM', 0.18421052631578946),
 ('ELROND', 0.09761583445793973),
 ('GOTHMOG', 0.09561403508771926),
 ('FARAMIR', 0.09412955465587046),
 ('FRODO', 0.0883940620782726),
 ('IORLAS', 0.05128205128205128),
 ('SMEAGOL', 0.05128205128205128),
 ('GIMLI', 0.04871794871794871)]

In [17]:
# Save the network.
nx.write_gexf(G, "LOTR1.gexf")

In [12]:
import collections
    
# create a subclass and override the handler methods
class MyHTMLSceneParser(HTMLParser):
    def __init__(self, characters):
        HTMLParser.__init__(self)
        self.is_b = False
        self.characters = characters
        self.lines = []
    
    def handle_starttag(self, tag, attrs):
        if tag == "b":
            self.is_b = True
        
    def handle_endtag(self, tag):
        if tag == "b":
            self.is_b = False
        
    def handle_data(self, data):
        if self.is_b:
            char = data.strip(" ,.·;:\\\"/-)")
            if char in self.characters:
                self.lines.append(char)

                

# Instantiate the parser and feed it the script.
parser = MyHTMLSceneParser(characters)
parser.feed(script)

counter = collections.Counter(parser.lines)
for elem in counter.most_common(10):
    print elem



('GANDALF', 67)
('SAM', 63)
('FRODO', 51)
('PIPPIN', 44)
('GOLLUM', 36)
('ARAGORN', 35)
('THEODEN', 28)
('DENETHOR', 22)
('EOWYN', 20)
('MERRY', 20)


In [13]:
import vincent
pre_data = counter.most_common(10)

vincent.core.initialize_notebook()
data = [x[1] for x in pre_data]
index = [x[0] for x in pre_data]
s = pd.Series(data=data, index=index)

bars = vincent.Bar(s)
bars.colors(brew='Pastel2')
bars.axis_titles(x='Character', y='# lines')
bars.scales['x'].padding = 0.1
bars.display()

In [14]:
# Character vs Number of scenes
all_scenes2 = []
for scene in all_scenes:
    ls = list(scene)
    all_scenes2.extend(ls)
    
counter = collections.Counter(all_scenes2)
for elem in counter.most_common(10):
    print elem


('GANDALF', 33)
('SAM', 23)
('PIPPIN', 23)
('FRODO', 21)
('ARAGORN', 17)
('GOLLUM', 16)
('THEODEN', 14)
('GIMLI', 11)
('EOWYN', 10)
('MERRY', 10)


In [15]:
import vincent
pre_data = counter.most_common(10)

vincent.core.initialize_notebook()
data = [x[1] for x in pre_data]
index = [x[0] for x in pre_data]
s = pd.Series(data=data, index=index)

bars = vincent.Bar(s)
bars.colors(brew='Pastel2')
bars.axis_titles(x='Character', y='# scenes')
bars.scales['x'].padding = 0.1
bars.display()